Versionamento git: requisitos
- python 3.11
- pip install gitpython

## Bibliotecas

In [ ]:

from git import Repo
import os, time
import pandas as pd

## Funções

In [ ]:

def set_path_current_repository() -> Repo:
    repo_path = os.path.realpath('.')
    repo = Repo(repo_path)
    if not repo.bare: print(f"Repositório encontrado: {repo_path}")
    else: print("Erro: Diretório não contém um repositório Git.")
    return repo


def add_all_files_to_commit(repo:Repo) :
    repo.git.add(all=True)


def commit_with_message(commit_message:str, repo:Repo):
    repo.index.commit(commit_message)


def git_push_all(repo:Repo):
    origin = repo.remote(name='origin')
    origin.push()


def print_last_commits(repo:Repo, max:int=10) -> pd.DataFrame:
    prev_commits = list(repo.iter_commits(all=True, max_count=max))
    last_commits = []
    for x in range(len(prev_commits)):
        commit_item = {
            'data_commit': str(prev_commits[x].committed_datetime)[:10],
            'msg_commit': str(prev_commits[x].message),
            'autor_commit': str(prev_commits[x].author)
        }
        last_commits.append(commit_item)
    df = pd.DataFrame(last_commits)
    return df


def print_files_from_git(repo:Repo, level=0):
    root = repo.head.commit.tree
    for entry in root:
        print(f'{"-" * 4 * level}| {entry.path}')
        if entry.type == "tree":
            print_files_from_git(entry, level + 1)


def consultar_tags(repo:Repo):
    todas_tags = list(repo.tags)
    if len(todas_tags) > 0:
        ultima_tag = todas_tags[-1]
        return str(ultima_tag.name)
    else:
        return str('nenhuma tag criada')
    

def set_initial_branchs(repo:Repo):
    all_branchs = [x.name for x in repo.branches]
    if 'main' not in all_branchs:
        repo.git.execute('git checkout -b main')

    if 'homolog' not in all_branchs:
        repo.git.execute('git checkout -b homolog')
        
    repo.git.checkout("homolog")

    time.sleep(2)
    repo.git.execute(f'git config init.defaultBranch main')
    repo.git.execute(f'git branch --set-upstream-to=origin/main main')
    repo.git.execute(f'git branch --set-upstream-to=origin/homolog homolog')


def criar_tag_e_atualizar_online(repo:Repo, tag_version:str, tag_message:str):
    repo.git.execute(f'git tag -a {tag_version} -m "{tag_message}"')
    repo.git.execute(f'git push origin {tag_version}')


def set_globals_config_git(repo:Repo):
    usuario_logado = 'edenilsonsantos' # os.getenv('username')
    email = 'santoeen@gmail.com' # f'{usuario_logado}@gmail.com'

    repo.git.execute(f'git config --global user.name "{usuario_logado}"')
    repo.git.execute(f'git config --global user.email "{email}"')


def commit_current_branch(repo:Repo, msg_commit:str, push_all=False):
    add_all_files_to_commit(repo)
    commit_with_message(msg_commit, repo)
    current_branch = str(repo.active_branch.name)
    repo.git.checkout(current_branch)
    time.sleep(3)
    if push_all:
        git_push_all(repo)



def create_new_feature(repo:Repo, feature_name:str='dev'):
    new_branch = repo.create_head(feature_name)
    new_branch.checkout()


def finish_feature(repo:Repo, feature_name:str='dev', merge_to:str='homolog', delete_feature:bool=True, push:bool=True):
    repo.git.checkout(merge_to) # esta sera a branch destino
    repo.git.merge(feature_name) # esta sera a branch origem
    if push:
        repo.git.push("origin", merge_to)
    if (delete_feature) and (feature_name not in ['main', 'homolog']):
        repo.git.branch("-d", feature_name)



def create_release(repo:Repo, release_version:str='1.0.0'):
    release_branch = repo.create_head(f"release-{release_version}")
    release_branch.checkout()


def finish_release(repo:Repo, release_version:str='1.0.0', delete_release:bool=True, push_all:bool=True):
    release_name = f"release-{release_version}"
    repo.git.checkout("main")
    repo.git.merge(release_name)
    repo.create_tag(release_version)
    repo.git.checkout("homolog")
    repo.git.merge(release_name)
    
    if (delete_release) and (release_name not in ['main', 'homolog']):
        repo.git.branch("-d", release_name)

    if push_all:
        for branch in repo.branches:
            branch.checkout()
            repo.git.push("origin", str(branch.name))  # Push para o remoto "origin"


def merge_branch(repo:Repo, branch_origem:str, branch_destino:str):
    all_branchs = [x.name for x in repo.branches]
    if branch_destino not in all_branchs:
        repo.git.execute(f'git checkout -b {branch_destino}')
    repo.git.checkout(branch_destino)
    repo.git.merge(branch_origem)
    repo.git.push("origin", branch_destino)



def delete_branch(repo:Repo, branch_name:str):
    atual = str(repo.active_branch.name)
    # fazer checkout, sair da branch que pretende excluir
    if branch_name == atual:
        all_branchs = [x.name for x in repo.branches]
        for x in all_branchs:
            if x not in branch_name:
                repo.git.checkout(x)
    repo.git.branch("-d", branch_name) # deletar a branch





## Execução

In [ ]:
# Preparar ambiente
repo = set_path_current_repository()
set_globals_config_git(repo)
set_initial_branchs(repo)

v1.0.0


In [ ]:
# Enviar commit e push all, branch atual
commit_current_branch(repo, 'add novas funcoes de comandos git', push_all=True)

In [ ]:
# Criar feature
create_new_feature(repo, 'dev')

In [ ]:
commit_current_branch(repo, 'versao testes feature dev', push_all=True)
finish_feature(repo, 'dev')

[<git.Head "refs/heads/homolog">, <git.Head "refs/heads/main">]